Jelmer Alphenaar 10655751, Bob van den Hoogen

In [9]:
from pprint import pprint
import numpy as np
import pycosat
import itertools
import csv
import time;

def sum_permutations(summers_needed, sum_total, exclude_value):
    l = [1,2,3,4,5,6,7,8,9]
    if(exclude_value != []):
        l = exclude_value
    nums = np.array(l)
    
    all_perms = list(itertools.combinations(nums, summers_needed))
    sum_perms = [p for p in all_perms if sum(p) == sum_total]
    return sum_perms

def load_sum_rules(blocks, indices_amount):
    rules = []

    for i in range(indices_amount):
        node_rule = []

        (summers, sum_total) = blocks[i]
        summers_needed = len(summers)
        perms = sum_permutations(len(summers), sum_total, [])

        for j in range(0, len(perms)):
            currentPerm = perms[j]
            for k in currentPerm:
                rule = str(i) + str(k)
                node_rule.append(rule)
        rules.append(node_rule)

    return rules

def load_blocks(filename, indices):
    blocks = {}
    with open(filename) as f:
        reader = csv.reader(f, delimiter=',')

        for line in reader:
            sum_list = [int(i) for i in line]
            block_sum = sum_list[-1]
            summers = sum_list[0:-1]
            for summer in summers:
                blocks[summer] = (summers, block_sum)

    return blocks

#convert rules to correct cnf
def convert_rules(rules):
    res = []
    for rule in rules:
        single_clause = []
        for clause in range(0, len(rule)):
            if len(rule[clause]) > 2:
                if rule[clause][0] != '-':
                    single_clause.append(v(int(str(rule[clause])[:-1]) // 9 + 1, int(str(rule[clause])[:-1]) % 9 + 1,
                        int(str(rule[clause])[len(str(rule[clause])) - 1])))
                else:
                    single_clause.append(-v(int(str(rule[clause])[1:-1]) // 9 + 1, int(str(rule[clause])[1:-1]) % 9 + 1,
                                           int(str(rule[clause])[len(str(rule[clause])) - 1])))
            else:
                single_clause.append(v(int(rule[clause][0]) // 9 + 1, int(rule[clause][0]) % 9 + 1, int(rule[clause][1])))
        res.append(single_clause)
    return res

#zet de sudoku om naar de juiste vorm
def sudoku_form2(x):
    empty_sudoku = [[0] * 9 for j in range(9)]
    for i in range(0, 81):
        empty_sudoku[int(i / 9)][i % 9] = x[int(i / 9)][i % 9]
    return empty_sudoku

# deze functie lijkt door de komst van versie 2 overbodig, afhankelijk van de inputvorm van de sudoku
def sudoku_form(x):
    test_sudoku = [[0]* 9 for j in range(9)]
    for i in range(0, 81):
        test_sudoku[int(i / 9)][i % 9] = int(x[i])
    return  test_sudoku

def load_implication_rules(blocks, indices):
    implication_rules = []

    for i in range(indices):
        (summers, sum_total) = blocks[i]
        if (len(summers) == 1):
            continue
            
        perms = sum_permutations(len(summers), sum_total, [])
        perm_list = []
        for perm in perms:
            for value in perm:
                if (not value in perm_list):
                    perm_list.append(value)
                    
#         if(len(summers) == 3):
#             for possible_value in perm_list:
#                 rule = ["-" + str(i) + str(possible_value)]
#                 summers_copy = list(summers)
#                 summers_copy.remove(i)
#                 other_sum = sum_total - possible_value
#                 perm_list_copy = list(perm_list)
#                 perm_list_copy.remove(possible_value)
#                 other_perms = sum_permutations(len(summers)-1, other_sum, perm_list_copy)
#                 other_perm_list = []
#                 for other_perm in other_perms:
#                     for value in other_perm:
#                         if (not value in other_perm_list):
#                             other_perm_list.append(value)
                
#                 for summer in summers_copy:
#                     for other_possible_value in other_perm_list:
#                         rule_copy = list(rule)
#                         rule_copy.append("-" + str(summer) + str(other_possible_value))
#                         last_summers_copy = list(summers_copy)
#                         last_summers_copy.remove(summer)
#                         last_perm_copy = list(other_perm_list)
#                         last_perm_copy.remove(other_possible_value)
#                         rule_copy.append(str(last_summers_copy[0]) + str(last_perm_copy[0]))
#                         implication_rules.append(rule_copy)6

        if(len(summers)>1 and len(summers) < 4):
            for possible_value in perm_list:
                other_sum = sum_total - possible_value
                print()
                print(sum_total)
                print(possible_value)
                print(other_sum)
                perm_list_copy = list(perm_list)
                perm_list_copy.remove(possible_value)
                other_perms = sum_permutations(len(summers)-1, other_sum, perm_list_copy)
            
                other_perm_list = []
                for other_perm in other_perms:
                    for value in other_perm:
                        if (not value in other_perm_list):
                            other_perm_list.append(value)
                
                print(other_perm_list)
                summers_copy = list(summers)
                summers_copy.remove(i)

                elements = []
                for summer in summers_copy:
                    elements_by_coord = []
                    #rule_copy = list(rule)
                    for other_perm in other_perm_list:
                        element = str(summer) + str(other_perm)
                        elements_by_coord.append(element)
                    elements.append(elements_by_coord)
                implied_elements_list = list(itertools.product(*elements))
                for implied_elements in implied_elements_list:
                    rule = ["-" + str(i) + str(possible_value)]
                    for implied_element in implied_elements:
                        rule.append(implied_element)
                    implication_rules.append(rule)
        
#         elements = []
#         for value in perm_list:
#             element = str(i) + str(value)
#             elements.append(element)

#         for element in elements:
#             implied_elements = list(elements)
#             implied_elements.remove(element)

#             rule = ["-" + element]
#             for implied_element in implied_elements:
#                 rule_copy = list(rule)
#                 rule_copy.append("-" + implied_element)
#                 implication_rules.append(rule_copy)

#         for value in perm_list:
#             rule = ["-" + str(i) + str(value)]
#             summers_copy = list(summers)
#             summers_copy.remove(i)
#             for summer in summers_copy:
#                 rule_copy = list(rule)
#                 rule_copy.append("-" + str(summer) + str(value))
#                 implication_rules.append(rule_copy)
#             rule = []
    print(implication_rules)
    return implication_rules

def v(i, j, d):
    """
    Return the number of the variable of cell i, j and digit d,
    which is an integer in the range of 1 to 729 (including).
    """
    return 81 * (i - 1) + 9 * (j - 1) + d

def standard_sudoku_clauses():
    """
    Create the (11745) Sudoku clauses, and return them as a list.
    Note that these clauses are *independent* of the particular
    Sudoku puzzle at hand.
    """
    res = []
    # for all cells, ensure that the each cell:
    for i in range(1, 10):
        for j in range(1, 10):
            # denotes (at least) one of the 9 digits (1 clause)
            res.append([v(i, j, d) for d in range(1, 10)])
            # does not denote two different digits at once (36 clauses)
            for d in range(1, 10):
                for dp in range(d + 1, 10):
                    res.append([-v(i, j, d), -v(i, j, dp)])


    def valid(cells):
        # Append 324 clauses, corresponding to 9 cells, to the result.
        # The 9 cells are represented by a list tuples.  The new clauses
        # ensure that the cells contain distinct values.
        for i, xi in enumerate(cells):
            for j, xj in enumerate(cells):
                if i < j:
                    for d in range(1, 10):
                        res.append([-v(xi[0], xi[1], d), -v(xj[0], xj[1], d)])

    # ensure rows and columns have distinct values
    for i in range(1, 10):
        valid([(i, j) for j in range(1, 10)])
        valid([(j, i) for j in range(1, 10)])
    # ensure 3x3 sub-grids "regions" have distinct values
    for i in 1, 4, 7:
        for j in 1, 4 ,7:
            valid([(i + k % 3, j + k // 3) for k in range(9)])

    assert len(res) == 81 * (1 + 36) + 27 * 324
    return res

def collect_clauses(grid, filename):
    """
    solve a Sudoku grid inplace
    """
    clauses = standard_sudoku_clauses()
    for i in range(1, 10):
        for j in range(1, 10):
            d = grid[i - 1][j - 1]
            # For each digit already known, a clause (with one literal).
            # Note:
            #     We could also remove all variables for the known cells
            #     altogether (which would be more efficient).  However, for
            #     the sake of simplicity, we decided not to do that.
            if d:
                clauses.append([v(i, j, d)])

    # load all information about the new sudoku
    indices_amount = len(new_sudoku) * len(new_sudoku)
    blocks = load_blocks(filename, indices_amount)
    sum_rules = convert_rules(load_sum_rules(blocks, indices_amount))
    killer_sudoku_clauses = convert_rules(load_implication_rules(blocks, indices_amount))

    # add all clauses in correct form
    for i in range(0, len(killer_sudoku_clauses)):
        clauses.append(killer_sudoku_clauses[i])

    # add all clauses in correct form
    for i in range(0, len(sum_rules)):
        clauses.append(sum_rules[i])
    
    return clauses, grid

def solve(grid, clauses):
    #sols = []
    
    #for sol in pycosat.itersolve(clauses):
    #    sols.append(set(sol))
    # solve the SAT problem
    sol = set(pycosat.solve(clauses))
    
    return sol

def sol2grid(sol):
#     grids = []
#     for sol in sols:
    for i in range(1, 10):
        for j in range(1, 10):
            grid[i - 1][j - 1] = read_cell(i, j, sol)
    #grids.append(grid)
    return grid
            
def read_cell(i, j, sol):
# return the digit of cell i, j according to the solution
    for d in range(1, 10):
        if v(i, j, d) in sol:
            return d

def check_solution(grid, sudokumatrix):
    npgrid = np.array(grid)
    if(np.array_equal(npgrid, sudokumatrix)):
        return 1
    else:
        return 0
        
rule_files = ["./easy_killer_sudoku1_rules.txt","./easy_killer_sudoku2_rules.txt","./easy_killer_sudoku3_rules.txt","./easy_killer_sudoku4_rules.txt","./easy_killer_sudoku5_rules.txt","./insane_killer_sudoku1_rules.txt","./insane_killer_sudoku2_rules.txt","./insane_killer_sudoku3_rules.txt","./insane_killer_sudoku4_rules.txt","./insane_killer_sudoku5_rules.txt",]    

empty_sudoku = [[0] * 9 for j in range(9)]
new_sudoku = sudoku_form2(empty_sudoku)
#print(file)
clauses, grid = collect_clauses(new_sudoku, "./insane_killer_sudoku1_rules.txt")
#print(clauses)
print(len(clauses))

#loop over all sudokus
loop_count = 1
avg_time = {}
count = 1
correct_solution = {}

# avg_block_size = {}
# block_count_dict = {}
# count2 = 1

# def file_len(fname):
#     with open(fname) as f:
#         for i, l in enumerate(f):
#             pass
#     return i + 1

# for file in rule_files:
#     empty_sudoku = [[0] * 9 for j in range(9)]
#     new_sudoku = sudoku_form2(empty_sudoku)
#     #print(file)
#     clauses, grid = collect_clauses(new_sudoku, file)
#     print(len(clauses))
#     print(clauses)
#     blocks = load_blocks(file, 81)  
#     block_count = 0
#     summers_sum = 0
    
#     for _, (summers, sum_total) in blocks.items():
#         block_count += 1
#         summers_sum += len(summers)
#     block_count_dict[count2] = file_len(file)
#     avg_block_size[count2] = summers_sum / float(block_count)
#     count2 += 1
# pprint (avg_block_size)
# pprint(block_count_dict)

# for file in rule_files:
#     total_cycles = 0
#     total_time = 0
#     for i in range(loop_count):
#         #nieuwe lege sudoku
#         empty_sudoku = [[0] * 9 for j in range(9)]
#         new_sudoku = sudoku_form2(empty_sudoku)
#         #print(file)
#         clauses, grid = collect_clauses(new_sudoku, file)
#         ms_begin = time.time()
#         solved_sudoku = solve(grid, clauses)
#         ms_end = time.time()
#         time_taken = ms_end - ms_begin
        
#         grid = sol2grid(solved_sudoku)

#         sudokumatrix = np.loadtxt(file.replace("rules", "solution"), dtype=int, delimiter=',')
#         correct_solution[count] = check_solution(grid, sudokumatrix)
        
#         total_time += time_taken
    
        
#     current_avg_time = total_time / float(loop_count)
#     avg_time[count] = current_avg_time
#     count += 1
# print(avg_time)
# print(correct_solution)


11
2
9
[9]

11
9
2
[2]

11
3
8
[8]

11
8
3
[3]

11
4
7
[7]

11
7
4
[4]

11
5
6
[6]

11
6
5
[5]

11
2
9
[9]

11
9
2
[2]

11
3
8
[8]

11
8
3
[3]

11
4
7
[7]

11
7
4
[4]

11
5
6
[6]

11
6
5
[5]

12
3
9
[9]

12
9
3
[3]

12
4
8
[8]

12
8
4
[4]

12
5
7
[7]

12
7
5
[5]

12
3
9
[9]

12
9
3
[3]

12
4
8
[8]

12
8
4
[4]

12
5
7
[7]

12
7
5
[5]

12
3
9
[9]

12
9
3
[3]

12
4
8
[8]

12
8
4
[4]

12
5
7
[7]

12
7
5
[5]

12
3
9
[9]

12
9
3
[3]

12
4
8
[8]

12
8
4
[4]

12
5
7
[7]

12
7
5
[5]

17
1
16
[7, 9]

17
7
10
[1, 9, 2, 8, 6, 4]

17
9
8
[1, 7, 2, 6, 3, 5]

17
2
15
[7, 8, 9, 6]

17
6
11
[7, 4, 9, 2, 8, 3]

17
8
9
[7, 2, 6, 3, 5, 4]

17
3
14
[9, 5, 6, 8]

17
5
12
[9, 3, 8, 4]

17
4
13
[7, 6, 8, 5]

17
1
16
[7, 9]

17
7
10
[1, 9, 2, 8, 6, 4]

17
9
8
[1, 7, 2, 6, 3, 5]

17
2
15
[7, 8, 9, 6]

17
6
11
[7, 4, 9, 2, 8, 3]

17
8
9
[7, 2, 6, 3, 5, 4]

17
3
14
[9, 5, 6, 8]

17
5
12
[9, 3, 8, 4]

17
4
13
[7, 6, 8, 5]

10
1
9
[2, 7, 3, 6, 4, 5]

10
2
8
[1, 7, 3, 5]

10
7
3
[1, 2]

10
3
7
[1, 6, 2, 5]

10
6
4
